# 🤖 Agentes RAG con OpenAI
## Clase de Apertura - Lazarus

En esta demostración veremos cómo crear un agente inteligente que:
1. Lee documentos
2. Busca información relevante
3. Responde preguntas con fuentes verificadas

## 1️⃣ Instalación de dependencias

Ejecuta esta celda una sola vez

In [6]:
# Instalamos las librerías necesarias
!pip install openai python-dotenv langchain langchain-openai langchain-community pypdf faiss-cpu -q

## 2️⃣ Configuración y carga de dependencias

In [7]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain_classic.callbacks import get_openai_callback

# Cargamos variables de entorno
load_dotenv()

# Verificamos que tenemos la API key
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("⚠️ No se encontró OPENAI_API_KEY. Asegúrate de crear un archivo .env con tu clave")

print("✅ API Key cargada correctamente")
print("✅ Todas las dependencias importadas")

✅ API Key cargada correctamente
✅ Todas las dependencias importadas


## 3️⃣ Configuración centralizada

In [8]:
# ⚙️ CONFIGURACIÓN CENTRALIZADA
# Modifica estos valores según tus necesidades

# Ruta al documento PDF
PDF_PATH = 'Catalogo_Equipos_Construccion.pdf'

# Configuración del text splitter
CHUNK_SIZE = 500
CHUNK_OVERLAP = 100

# Configuración del modelo OpenAI
MODEL_NAME = "gpt-4o-mini"
TEMPERATURE = 0.3  # Más bajo = respuestas más consistentes
MAX_TOKENS = 500

# Configuración del retriever
TOP_K_DOCUMENTS = 3  # Cuántos documentos recuperar

# Modelo de embeddings
EMBEDDING_MODEL = "text-embedding-3-small"

# Directorio para persistencia
VECTORSTORE_DIR = "vectorstore_db"

print("✅ Configuración cargada:")
print(f"   - PDF: {PDF_PATH}")
print(f"   - Chunk size: {CHUNK_SIZE} | Overlap: {CHUNK_OVERLAP}")
print(f"   - Modelo: {MODEL_NAME} | Temp: {TEMPERATURE}")
print(f"   - Embeddings: {EMBEDDING_MODEL}")
print(f"   - Top-K docs: {TOP_K_DOCUMENTS}")

✅ Configuración cargada:
   - PDF: Catalogo_Equipos_Construccion.pdf
   - Chunk size: 500 | Overlap: 100
   - Modelo: gpt-4o-mini | Temp: 0.3
   - Embeddings: text-embedding-3-small
   - Top-K docs: 3


## 4️⃣ Paso 1 - RETRIEVAL: Cargamos y vectorizamos el documento

In [9]:
print("📚 PASO 1: RETRIEVAL - Procesando documento...\n")

# Validamos que el archivo existe
if not os.path.exists(PDF_PATH):
    raise FileNotFoundError(f"❌ No se encontró el archivo: {PDF_PATH}")

try:
    # Cargamos el documento PDF
    loader = PyPDFLoader(PDF_PATH)
    documents = loader.load()
    print(f"✅ Documento cargado: {len(documents)} página(s)")
    
    # Verificamos que haya contenido
    total_chars = sum(len(doc.page_content) for doc in documents)
    if total_chars == 0:
        raise ValueError("❌ El PDF no contiene texto extraíble. Puede ser un PDF escaneado que requiere OCR.")
    
    print(f"✅ Total de caracteres: {total_chars}")
    
    # Dividimos el documento en chunks usando RecursiveCharacterTextSplitter
    # Este splitter es mejor porque mantiene el contexto semántico
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separators=["\n\n", "\n", " ", ""]
    )
    docs = text_splitter.split_documents(documents)
    
    if len(docs) == 0:
        raise ValueError("❌ No se pudieron crear chunks. El documento puede ser demasiado corto.")
    
    print(f"✅ Documento dividido en {len(docs)} chunks")
    
    # Mostramos ejemplos de chunks creados
    print(f"\n📋 Ejemplo de chunk 1:")
    print(f"   {docs[0].page_content[:200]}...")
    if len(docs) > 1:
        print(f"\n📋 Ejemplo de chunk 2:")
        print(f"   {docs[1].page_content[:200]}...")
    
    # Creamos embeddings (convertimos texto a vectores)
    print(f"\n🔄 Creando embeddings con {EMBEDDING_MODEL}...")
    embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    
    # Verificamos si existe el vectorstore guardado
    if os.path.exists(VECTORSTORE_DIR):
        print(f"📦 Cargando vectorstore existente desde {VECTORSTORE_DIR}...")
        vectorstore = FAISS.load_local(VECTORSTORE_DIR, embeddings, allow_dangerous_deserialization=True)
        print("✅ Vectorstore cargado desde disco (ahorro de costos)")
    else:
        print("🔄 Creando nueva base de datos vectorial...")
        vectorstore = FAISS.from_documents(docs, embeddings)
        # Guardamos para uso futuro
        vectorstore.save_local(VECTORSTORE_DIR)
        print(f"✅ Base de datos vectorial creada y guardada en {VECTORSTORE_DIR}")
    
    # Creamos el retriever con búsqueda por similaridad y scores
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": TOP_K_DOCUMENTS}
    )
    print(f"✅ Retriever configurado para buscar top {TOP_K_DOCUMENTS} documentos")
    
except FileNotFoundError as e:
    print(f"❌ {str(e)}")
    raise
except ValueError as e:
    print(f"❌ {str(e)}")
    raise
except Exception as e:
    print(f"❌ Error inesperado al procesar el documento: {str(e)}")
    raise

📚 PASO 1: RETRIEVAL - Procesando documento...



✅ Documento cargado: 7 página(s)
✅ Total de caracteres: 8548
✅ Documento dividido en 22 chunks

📋 Ejemplo de chunk 1:
   Catalogo de Equipos de Construccion - Lazarus Rental
EQUIPOS PARA CONCRETO
BAILARINA COMPACTADORA
Precio: L850.00 por dia
Es ideal para llevar a cabo trabajos de compactacion en zanjas u otros espacio...

📋 Ejemplo de chunk 2:
   Capacidad de 560 libras.
Disponibilidad: En stock
COMPACTADOR DE RODILLO 524 KGS 1 EJE (TORITO)
Precio: L2,000.00 por dia
Disenado para la compactacion de bases granulares, almacenes, estacionamientos...

🔄 Creando embeddings con text-embedding-3-small...
🔄 Creando nueva base de datos vectorial...
🔄 Creando nueva base de datos vectorial...
✅ Base de datos vectorial creada y guardada en vectorstore_db
✅ Retriever configurado para buscar top 3 documentos
✅ Base de datos vectorial creada y guardada en vectorstore_db
✅ Retriever configurado para buscar top 3 documentos


## 5️⃣ Paso 2 - GENERATION: Configuramos el modelo OpenAI

In [10]:
print("\n🤖 PASO 2: GENERATION - Configurando modelo OpenAI...\n")

llm = ChatOpenAI(
    model=MODEL_NAME,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS
)

print(f"✅ Modelo OpenAI ({MODEL_NAME}) configurado")
print(f"   - Temperatura: {TEMPERATURE} (respuestas más consistentes y precisas)")
print(f"   - Max tokens: {MAX_TOKENS} (límite de respuesta)")
print("\n💡 Temperatura baja (0.3) es ideal para atención al cliente")


🤖 PASO 2: GENERATION - Configurando modelo OpenAI...

✅ Modelo OpenAI (gpt-4o-mini) configurado
   - Temperatura: 0.3 (respuestas más consistentes y precisas)
   - Max tokens: 500 (límite de respuesta)

💡 Temperatura baja (0.3) es ideal para atención al cliente
✅ Modelo OpenAI (gpt-4o-mini) configurado
   - Temperatura: 0.3 (respuestas más consistentes y precisas)
   - Max tokens: 500 (límite de respuesta)

💡 Temperatura baja (0.3) es ideal para atención al cliente


## 6️⃣ Paso 3 - PROMPT ENGINEERING: Creamos el prompt del agente

In [11]:
print("\n📝 PASO 3: PROMPT ENGINEERING - Diseñando instrucciones...\n")

# Creamos un prompt template personalizado
prompt_template = """Eres un agente de atención al cliente para CONCESA, especializado en equipos de construcción.

Usa la siguiente información de contexto para responder:

{context}

Pregunta: {question}

Instrucciones importantes:
1. Si te preguntan quién eres o sobre tu rol: Responde que eres un asistente virtual de CONCESA especializado en productos y equipos de construcción, diseñado para ayudar a los clientes con información sobre el catálogo de productos.
2. Para preguntas sobre productos: Usa ÚNICAMENTE la información del contexto proporcionado.
3. Si la pregunta es sobre temas NO relacionados con productos o tu rol (ej: geografía, historia, matemáticas), responde: "Lo siento, esa pregunta está fuera de mi área de conocimiento. Solo puedo responder preguntas sobre nuestros productos y mi función como asistente."
4. Si no encuentras información sobre un producto específico en el contexto, responde: "Lo siento, no tengo información sobre ese producto en mi base de datos actual"
5. Sé específico y referencia la información del documento cuando hables de productos
6. Sé profesional, amable y conciso
7. Responde únicamente en español

Respuesta:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

print("✅ Prompt template creado")
print("\nEste prompt:") 
print("- Define el rol del agente (atención al cliente de CONCESA)")
print("- Permite responder sobre su identidad y función")
print("- Restringe las respuestas de productos solo al PDF")
print("- Rechaza preguntas fuera de contexto educadamente")
print("- Incluye contexto relevante del documento")


📝 PASO 3: PROMPT ENGINEERING - Diseñando instrucciones...

✅ Prompt template creado

Este prompt:
- Define el rol del agente (atención al cliente de CONCESA)
- Permite responder sobre su identidad y función
- Restringe las respuestas de productos solo al PDF
- Rechaza preguntas fuera de contexto educadamente
- Incluye contexto relevante del documento


## 7️⃣ Paso 4 - ORQUESTACIÓN: Creamos la cadena RAG

In [12]:
print("\n⚙️  PASO 4: ORQUESTACIÓN - Conectando retrieval + generation...\n")

# Creamos la cadena RAG que combina retrieval + generation
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

print("✅ Cadena RAG creada")
print("\nCómo funciona:")
print("1. Usuario hace una pregunta")
print("2. Retriever busca documentos relevantes")
print("3. LLM genera respuesta basada en esos documentos")
print("4. Devolvemos respuesta + fuentes")


⚙️  PASO 4: ORQUESTACIÓN - Conectando retrieval + generation...

✅ Cadena RAG creada

Cómo funciona:
1. Usuario hace una pregunta
2. Retriever busca documentos relevantes
3. LLM genera respuesta basada en esos documentos
4. Devolvemos respuesta + fuentes


## 8️⃣ ¡DEMO EN VIVO! Prueba 1: Pregunta sobre productos

In [13]:
print("\n" + "="*60)
print("🔴 DEMO EN VIVO - PRUEBA 1")
print("="*60)

pregunta_1 = "Necesito un demoledor para 50 pastillas de concreto"
print(f"\n📌 Pregunta: {pregunta_1}\n")

# Usamos callback para tracking de costos
with get_openai_callback() as cb:
    resultado_1 = qa_chain({"query": pregunta_1})
    
    print("\n🤖 RESPUESTA DEL AGENTE:")
    print("-" * 60)
    print(resultado_1["result"])
    print("-" * 60)
    
    # Mostramos documentos con scores de similaridad
    print("\n📄 DOCUMENTOS FUENTE UTILIZADOS:")
    docs_with_scores = vectorstore.similarity_search_with_score(pregunta_1, k=TOP_K_DOCUMENTS)
    for i, (doc, score) in enumerate(docs_with_scores, 1):
        print(f"\n  Fuente {i} (Similaridad: {score:.4f}):")
        print(f"  {doc.page_content[:200]}...")
    
    # Mostramos métricas de costo
    print(f"\n💰 MÉTRICAS DE COSTO:")
    print(f"   - Tokens totales: {cb.total_tokens}")
    print(f"   - Tokens prompt: {cb.prompt_tokens}")
    print(f"   - Tokens respuesta: {cb.completion_tokens}")
    print(f"   - Costo total: ${cb.total_cost:.6f} USD")


🔴 DEMO EN VIVO - PRUEBA 1

📌 Pregunta: Necesito un demoledor para 50 pastillas de concreto


🔴 DEMO EN VIVO - PRUEBA 1

📌 Pregunta: Necesito un demoledor para 50 pastillas de concreto



C:\Users\angel\AppData\Local\Temp\ipykernel_16556\106705076.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  resultado_1 = qa_chain({"query": pregunta_1})



🤖 RESPUESTA DEL AGENTE:
------------------------------------------------------------
Para demoler 50 pastillas de concreto, te recomendaría considerar el uso del DEMOLEDOR TE-3000, ya que es un martillo rompedor excepcionalmente potente, ideal para tareas pesadas de demolición de concreto. Su precio es de L1,100.00 por día. 

Si necesitas un equipo más versátil, también podrías optar por el DEMOLEDOR TE-1000, que es adecuado para tareas de rotura de suelos y aplicaciones ocasionales de paredes, con un costo de L800.00 por día.

Ambos equipos están disponibles en stock. Si necesitas más información o deseas realizar una reserva, no dudes en decírmelo.
------------------------------------------------------------

📄 DOCUMENTOS FUENTE UTILIZADOS:

  Fuente 1 (Similaridad: 0.8777):
  Disponibilidad: En stock
DEMOLEDOR TE-800
Precio: L550.00 por dia
Martillo demoledor de alto rendimiento y robusto para trabajar en paredes y pisos. Marca HILTI modelo TE-800.
Disponibilidad: En stock...

  Fu

# 🚀 SISTEMA RAG CON CHAT + HISTORIAL + TOOLS

## Integración completa: Conversación persistente + Documentos + Funciones

##  Imports adicionales para el chat con tools

In [19]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from datetime import datetime, timedelta

print("✅ Imports adicionales cargados para chat con tools")

✅ Imports adicionales cargados para chat con tools


## Definición de Tools (Funciones del agente)

In [20]:
# 🔧 HERRAMIENTA 1: Buscar información en el catálogo PDF
@tool
def buscar_info_producto(producto: str) -> str:
    """Busca información de productos en el catálogo (precios, especificaciones, disponibilidad).
    
    Args:
        producto: Nombre del producto a buscar (ej: 'rotomartillo', 'compactador')
    """
    docs = retriever.invoke(producto)
    if docs:
        return docs[0].page_content[:500]
    return "No se encontró información del producto"

# 🔧 HERRAMIENTA 2: Calcular descuentos por volumen
@tool
def calcular_descuento(precio: float, dias: int) -> dict:
    """Calcula el precio total de renta con descuentos por volumen.
    
    Args:
        precio: Precio por día en Lempiras (ej: 550.00)
        dias: Número de días de renta (ej: 15)
    
    Descuentos:
    - 7-13 días: 10% descuento
    - 14-29 días: 15% descuento
    - 30+ días: 20% descuento
    """
    total = precio * dias
    
    if dias >= 30:
        descuento = 0.20
    elif dias >= 14:
        descuento = 0.15
    elif dias >= 7:
        descuento = 0.10
    else:
        descuento = 0
    
    total_con_descuento = total * (1 - descuento)
    ahorro = total - total_con_descuento
    
    return {
        "total_sin_descuento": f"L{total:,.2f}",
        "descuento_porcentaje": f"{descuento*100:.0f}%",
        "total_con_descuento": f"L{total_con_descuento:,.2f}",
        "ahorro": f"L{ahorro:,.2f}"
    }

# 🔧 HERRAMIENTA 3: Verificar disponibilidad en inventario
@tool
def verificar_disponibilidad(equipo: str) -> str:
    """Verifica si un equipo está disponible en inventario actual.
    
    Args:
        equipo: Nombre del equipo (ej: 'demoledor', 'rotomartillo')
    """
    # Base de datos simulada de inventario
    inventario = {
        "demoledor": {"disponible": True, "unidades": 3},
        "rotomartillo": {"disponible": True, "unidades": 5},
        "compactador": {"disponible": False, "unidades": 0},
        "bailarina": {"disponible": True, "unidades": 2}
    }
    
    for key, info in inventario.items():
        if key in equipo.lower():
            if info["disponible"]:
                return f"✅ {equipo.upper()} disponible. Stock actual: {info['unidades']} unidades"
            else:
                return f"❌ {equipo.upper()} agotado temporalmente. Stock: {info['unidades']}"
    
    return f"⚠️ Equipo no encontrado en inventario: {equipo}"

# 🔧 HERRAMIENTA 4: Calcular fecha de devolución
@tool
def calcular_fecha_entrega(dias: int) -> str:
    """Calcula la fecha de devolución del equipo rentado.
    
    Args:
        dias: Número de días de renta
    """
    fecha_hoy = datetime.now()
    fecha_devolucion = fecha_hoy + timedelta(days=dias)
    
    return f"""📅 Fechas de renta:
- Inicio: {fecha_hoy.strftime('%d/%m/%Y')}
- Devolución: {fecha_devolucion.strftime('%d/%m/%Y')}
- Total días: {dias}"""

# Lista de todas las herramientas
tools = [buscar_info_producto, calcular_descuento, verificar_disponibilidad, calcular_fecha_entrega]

print("✅ Tools creadas exitosamente:")
for tool in tools:
    print(f"   - {tool.name}: {tool.description[:60]}...")

✅ Tools creadas exitosamente:
   - buscar_info_producto: Busca información de productos en el catálogo (precios, espe...
   - calcular_descuento: Calcula el precio total de renta con descuentos por volumen....
   - verificar_disponibilidad: Verifica si un equipo está disponible en inventario actual.
...
   - calcular_fecha_entrega: Calcula la fecha de devolución del equipo rentado.

    Args...


## Sistema de Chat con Historial de Mensajes

In [21]:
class ChatRAGConTools:
    """Sistema de chat que integra:
    - Historial de conversación
    - Consulta a documentos (RAG)
    - Funciones/Tools
    """
    
    def __init__(self, llm, tools, retriever):
        self.llm = llm.bind_tools(tools)
        self.tools = tools
        self.retriever = retriever
        self.historial = []  # Almacena toda la conversación
        self.max_iterations = 5
        
    def chat(self, mensaje_usuario: str) -> str:
        """Procesa un mensaje del usuario y devuelve la respuesta"""
        
        print(f"\n{'='*60}")
        print(f"💬 Usuario: {mensaje_usuario}")
        print(f"{'='*60}")
        
        # Agregar mensaje del usuario al historial
        self.historial.append(HumanMessage(content=mensaje_usuario))
        
        # Loop de iteraciones (para uso de múltiples tools)
        for iteracion in range(self.max_iterations):
            
            # LLM procesa el historial completo
            response = self.llm.invoke(self.historial)
            self.historial.append(response)
            
            # Si no hay tool calls, es la respuesta final
            if not response.tool_calls:
                print(f"\n🤖 Asistente: {response.content}\n")
                return response.content
            
            # Ejecutar tools solicitadas
            print(f"\n🔧 Iteración {iteracion + 1}: Usando herramientas...")
            for tool_call in response.tool_calls:
                tool_name = tool_call['name']
                tool_args = tool_call['args']
                
                print(f"   → {tool_name}({tool_args})")
                
                # Buscar y ejecutar la tool
                for tool in self.tools:
                    if tool.name == tool_name:
                        result = tool.invoke(tool_args)
                        print(f"   ✓ Resultado: {str(result)[:100]}...")
                        
                        # Agregar resultado al historial
                        self.historial.append(
                            ToolMessage(
                                content=str(result),
                                tool_call_id=tool_call['id']
                            )
                        )
        
        return "⚠️ Límite de iteraciones alcanzado"
    
    def ver_historial(self):
        """Muestra el historial de la conversación"""
        print(f"\n{'='*60}")
        print("📜 HISTORIAL DE CONVERSACIÓN")
        print(f"{'='*60}")
        
        contador = 0
        for msg in self.historial:
            if isinstance(msg, HumanMessage):
                contador += 1
                print(f"\n[{contador}] 👤 Usuario:")
                print(f"    {msg.content}")
            elif isinstance(msg, AIMessage):
                if msg.content:
                    print(f"\n[{contador}] 🤖 Asistente:")
                    print(f"    {msg.content}")
                if msg.tool_calls:
                    print(f"\n[{contador}] 🔧 Tools llamadas:")
                    for tc in msg.tool_calls:
                        print(f"    - {tc['name']}({tc['args']})")
            elif isinstance(msg, ToolMessage):
                print(f"\n[{contador}] 📊 Resultado:")
                print(f"    {msg.content[:150]}...")
        
        print(f"\n{'='*60}")
        print(f"Total de mensajes en historial: {len(self.historial)}")
        print(f"{'='*60}\n")
    
    def limpiar_historial(self):
        """Limpia el historial de conversación"""
        self.historial = []
        print("✅ Historial limpiado")

# Crear instancia del chat
chat_sistema = ChatRAGConTools(llm, tools, retriever)

print("✅ Sistema de chat con historial creado")
print("\nFunciones disponibles:")
print("  - chat_sistema.chat('tu pregunta')  → Enviar mensaje")
print("  - chat_sistema.ver_historial()      → Ver conversación")
print("  - chat_sistema.limpiar_historial()  → Limpiar chat")

✅ Sistema de chat con historial creado

Funciones disponibles:
  - chat_sistema.chat('tu pregunta')  → Enviar mensaje
  - chat_sistema.ver_historial()      → Ver conversación
  - chat_sistema.limpiar_historial()  → Limpiar chat


## 💬 DEMO: Conversación 1 - Consulta básica

In [22]:
# Primera pregunta
chat_sistema.chat("¿Qué información tienes sobre rotomartillos?")


💬 Usuario: ¿Qué información tienes sobre rotomartillos?

🔧 Iteración 1: Usando herramientas...
   → buscar_info_producto({'producto': 'rotomartillo'})

🔧 Iteración 1: Usando herramientas...
   → buscar_info_producto({'producto': 'rotomartillo'})
   ✓ Resultado: Disponibilidad: En stock
ROTOMARTILLO TE-70
Precio: L750.00 por dia
Martillo perforador muy potente ...
   ✓ Resultado: Disponibilidad: En stock
ROTOMARTILLO TE-70
Precio: L750.00 por dia
Martillo perforador muy potente ...

🤖 Asistente: Aquí tienes la información sobre los rotomartillos disponibles:

1. **ROTOMARTILLO TE-70**
   - **Precio:** L750.00 por día
   - **Descripción:** Martillo perforador muy potente para tareas de carga pesada de taladro y cincelado en concreto.
   - **Marca:** HILTI
   - **Modelo:** TE-70
   - **Disponibilidad:** En stock

2. **ROTOMARTILLO TE-500**
   - **Precio:** L320.00 por día
   - **Descripción:** Martillo rompedor versátil para tareas de cincelado ligero en hormigón y mampostería.
   - **Ma

'Aquí tienes la información sobre los rotomartillos disponibles:\n\n1. **ROTOMARTILLO TE-70**\n   - **Precio:** L750.00 por día\n   - **Descripción:** Martillo perforador muy potente para tareas de carga pesada de taladro y cincelado en concreto.\n   - **Marca:** HILTI\n   - **Modelo:** TE-70\n   - **Disponibilidad:** En stock\n\n2. **ROTOMARTILLO TE-500**\n   - **Precio:** L320.00 por día\n   - **Descripción:** Martillo rompedor versátil para tareas de cincelado ligero en hormigón y mampostería.\n   - **Marca:** HILTI\n   - **Modelo:** TE-500\n   - **Disponibilidad:** En stock\n\nSi necesitas más información o deseas realizar una renta, házmelo saber.'

## DEMO: Conversación 2 - Pregunta con contexto previo

In [30]:
# Segunda pregunta (hace referencia a la anterior)
chat_sistema.chat("¿Está disponible ese equipo?")


💬 Usuario: ¿Está disponible ese equipo?

🤖 Asistente: Por favor, indícame el nombre del equipo que deseas verificar para saber si está disponible.


🤖 Asistente: Por favor, indícame el nombre del equipo que deseas verificar para saber si está disponible.



'Por favor, indícame el nombre del equipo que deseas verificar para saber si está disponible.'

## DEMO: Conversación 3 - Cálculo de precios con herramienta

In [31]:
# Tercera pregunta (usa múltiples tools: buscar precio + calcular descuento)
chat_sistema.chat("¿Cuánto me costaría rentarlo por 15 días?")


💬 Usuario: ¿Cuánto me costaría rentarlo por 15 días?

🤖 Asistente: Para poder calcular el costo de la renta por 15 días, necesito saber el nombre del equipo que deseas rentar. ¿Podrías proporcionármelo?


🤖 Asistente: Para poder calcular el costo de la renta por 15 días, necesito saber el nombre del equipo que deseas rentar. ¿Podrías proporcionármelo?



'Para poder calcular el costo de la renta por 15 días, necesito saber el nombre del equipo que deseas rentar. ¿Podrías proporcionármelo?'

## 2️⃣0️⃣ 💬 DEMO: Conversación 4 - Fecha de entrega

In [25]:
# Cuarta pregunta (usa tool de fecha)
chat_sistema.chat("¿Cuándo tendría que devolverlo?")


💬 Usuario: ¿Cuándo tendría que devolverlo?

🔧 Iteración 1: Usando herramientas...
   → calcular_fecha_entrega({'dias': 15})
   ✓ Resultado: 📅 Fechas de renta:
- Inicio: 06/12/2025
- Devolución: 21/12/2025
- Total días: 15...
   → calcular_fecha_entrega({'dias': 15})
   ✓ Resultado: 📅 Fechas de renta:
- Inicio: 06/12/2025
- Devolución: 21/12/2025
- Total días: 15...

🔧 Iteración 1: Usando herramientas...
   → calcular_fecha_entrega({'dias': 15})
   ✓ Resultado: 📅 Fechas de renta:
- Inicio: 06/12/2025
- Devolución: 21/12/2025
- Total días: 15...
   → calcular_fecha_entrega({'dias': 15})
   ✓ Resultado: 📅 Fechas de renta:
- Inicio: 06/12/2025
- Devolución: 21/12/2025
- Total días: 15...

🤖 Asistente: Si rentas cualquiera de los rotomartillos por 15 días, las fechas serían las siguientes:

- **Fecha de inicio de la renta:** 06/12/2025
- **Fecha de devolución:** 21/12/2025
- **Total días de renta:** 15

Si necesitas más información o deseas proceder con la renta, no dudes en decírmelo.


🤖

'Si rentas cualquiera de los rotomartillos por 15 días, las fechas serían las siguientes:\n\n- **Fecha de inicio de la renta:** 06/12/2025\n- **Fecha de devolución:** 21/12/2025\n- **Total días de renta:** 15\n\nSi necesitas más información o deseas proceder con la renta, no dudes en decírmelo.'

## 2️⃣1️⃣ 📜 Ver el historial completo de la conversación

In [34]:
# Ver todo el historial de mensajes
chat_sistema.ver_historial()


📜 HISTORIAL DE CONVERSACIÓN

[1] 👤 Usuario:
    ¿Está disponible ese equipo?

[1] 🤖 Asistente:
    Por favor, indícame el nombre del equipo que deseas verificar para saber si está disponible.

[2] 👤 Usuario:
    ¿Cuánto me costaría rentarlo por 15 días?

[2] 🤖 Asistente:
    Para poder calcular el costo de la renta por 15 días, necesito saber el nombre del equipo que deseas rentar. ¿Podrías proporcionármelo?

[3] 👤 Usuario:
    Busco un ROTOMARTILLO TE-70

[3] 🔧 Tools llamadas:
    - verificar_disponibilidad({'equipo': 'rotomartillo TE-70'})
    - buscar_info_producto({'producto': 'rotomartillo TE-70'})

[3] 📊 Resultado:
    ✅ ROTOMARTILLO TE-70 disponible. Stock actual: 5 unidades...

[3] 📊 Resultado:
    Disponibilidad: En stock
ROTOMARTILLO TE-70
Precio: L750.00 por dia
Martillo perforador muy potente para tareas de carga pesada de taladro y cincelado...

[3] 🔧 Tools llamadas:
    - calcular_descuento({'precio': 750, 'dias': 15})

[3] 📊 Resultado:
    {'total_sin_descuento': 'L11,2

## Prueba tú mismo: Chat interactivo

In [27]:
# 🎮 ZONA DE PRUEBAS - Cambia la pregunta y ejecuta
# Ejemplos de preguntas que puedes hacer:

# Sobre productos:
# chat_sistema.chat("¿Qué equipos de demolición tienen?")
# chat_sistema.chat("Necesito información sobre compactadores")

# Con cálculos:
# chat_sistema.chat("¿Cuánto cuesta rentar un demoledor por 30 días?")
# chat_sistema.chat("Dame una cotización para la bailarina por 10 días")

# Disponibilidad:
# chat_sistema.chat("¿Tienen compactadores en stock?")

# Fechas:
# chat_sistema.chat("Si rento por 20 días, ¿cuándo debo devolver?")

# Conversacionales (usa el contexto previo):
# chat_sistema.chat("¿Y cuánto me ahorraría?")
# chat_sistema.chat("¿Hay otros equipos similares?")

# TU PREGUNTA AQUÍ:
chat_sistema.chat("¿Quién eres?")


💬 Usuario: ¿Quién eres?

🤖 Asistente: Soy un asistente virtual diseñado para ayudarte con información sobre productos, servicios y procesos de renta. Puedo proporcionarte detalles sobre equipos, precios, disponibilidad y otros aspectos relacionados. Si tienes alguna pregunta o necesitas asistencia, ¡estaré encantado de ayudarte!


🤖 Asistente: Soy un asistente virtual diseñado para ayudarte con información sobre productos, servicios y procesos de renta. Puedo proporcionarte detalles sobre equipos, precios, disponibilidad y otros aspectos relacionados. Si tienes alguna pregunta o necesitas asistencia, ¡estaré encantado de ayudarte!



'Soy un asistente virtual diseñado para ayudarte con información sobre productos, servicios y procesos de renta. Puedo proporcionarte detalles sobre equipos, precios, disponibilidad y otros aspectos relacionados. Si tienes alguna pregunta o necesitas asistencia, ¡estaré encantado de ayudarte!'

## Limpiar historial (empezar conversación nueva)

In [35]:
# Ejecuta esto cuando quieras empezar una conversación nueva
chat_sistema.limpiar_historial()

✅ Historial limpiado


## Resumen: ¿Qué logramos en esta sección?

In [36]:
print("\n" + "="*70)
print("✨ SISTEMA RAG CON CHAT + HISTORIAL + TOOLS")
print("="*70)

print("""
🎯 FUNCIONALIDADES IMPLEMENTADAS:

1️⃣ HISTORIAL DE CONVERSACIÓN
   ✅ Mantiene contexto de toda la conversación
   ✅ El agente recuerda preguntas y respuestas anteriores
   ✅ Puedes hacer preguntas de seguimiento sin repetir contexto
   
2️⃣ CONSULTA A DOCUMENTOS (RAG)
   ✅ Busca información en tu catálogo PDF
   ✅ Respuestas basadas en documentos reales
   ✅ Evita alucinaciones del LLM
   
3️⃣ HERRAMIENTAS (TOOLS/FUNCTIONS)
   ✅ buscar_info_producto → Busca en el PDF
   ✅ calcular_descuento → Calcula precios con descuentos
   ✅ verificar_disponibilidad → Revisa inventario
   ✅ calcular_fecha_entrega → Calcula fechas de devolución
   
4️⃣ RAZONAMIENTO MULTI-PASO
   ✅ Puede usar múltiples herramientas en una conversación
   ✅ Combina información de diferentes fuentes
   ✅ Toma decisiones inteligentes sobre qué tool usar

💡 EJEMPLO DE FLUJO COMPLETO:
   Usuario: "¿Qué información tienes sobre rotomartillos?"
   → Tool: buscar_info_producto
   
   Usuario: "¿Está disponible ese equipo?"
   → Usa contexto previo (rotomartillo)
   → Tool: verificar_disponibilidad
   
   Usuario: "¿Cuánto me costaría rentarlo por 15 días?"
   → Usa contexto previo (rotomartillo)
   → Tool 1: buscar_info_producto (obtener precio)
   → Tool 2: calcular_descuento (calcular total)
   
   Usuario: "¿Cuándo tendría que devolverlo?"
   → Usa contexto previo (15 días)
   → Tool: calcular_fecha_entrega

🔥 VENTAJAS vs CLASE ANTERIOR:
   ✅ Antes: Una pregunta → Una respuesta (sin memoria)
   ✅ Ahora: Conversación completa con contexto
   ✅ Antes: Solo consulta documentos
   ✅ Ahora: Documentos + Cálculos + Validaciones
   ✅ Antes: Respuestas estáticas
   ✅ Ahora: Acciones dinámicas con tools
""")

print("="*70)
print("🎓 ¡Sistema completo de chat conversacional con RAG y Tools!")
print("="*70)


✨ SISTEMA RAG CON CHAT + HISTORIAL + TOOLS

🎯 FUNCIONALIDADES IMPLEMENTADAS:

1️⃣ HISTORIAL DE CONVERSACIÓN
   ✅ Mantiene contexto de toda la conversación
   ✅ El agente recuerda preguntas y respuestas anteriores
   ✅ Puedes hacer preguntas de seguimiento sin repetir contexto
   
2️⃣ CONSULTA A DOCUMENTOS (RAG)
   ✅ Busca información en tu catálogo PDF
   ✅ Respuestas basadas en documentos reales
   ✅ Evita alucinaciones del LLM
   
3️⃣ HERRAMIENTAS (TOOLS/FUNCTIONS)
   ✅ buscar_info_producto → Busca en el PDF
   ✅ calcular_descuento → Calcula precios con descuentos
   ✅ verificar_disponibilidad → Revisa inventario
   ✅ calcular_fecha_entrega → Calcula fechas de devolución
   
4️⃣ RAZONAMIENTO MULTI-PASO
   ✅ Puede usar múltiples herramientas en una conversación
   ✅ Combina información de diferentes fuentes
   ✅ Toma decisiones inteligentes sobre qué tool usar

💡 EJEMPLO DE FLUJO COMPLETO:
   Usuario: "¿Qué información tienes sobre rotomartillos?"
   → Tool: buscar_info_producto
   
